# Chapter 3 - notebook level 1

## Save NLIW event data in ENU coordinates

In [1]:
import os
import numpy as np
import xarray as xr
import seaborn as sns
import cmocean as cm
import natsort
import glob
# from d2spike.despike import DataArray
from wootils.plotnice import vert_stack, basic_ts#, saveclose
# from pIMOS.utils.nortek_signature_utils import beam2inst, inst2earth

import sys
sys.path.insert(0, '../../')
# from chp3_despike import full_pipe
from chp3_basefuncs import get_sigfiles
from chp3_fluxfuncs import get_nliw_times

sns.set()
sns.set_theme(style="white")
sns.set_style("ticks")

import warnings
warnings.filterwarnings("ignore")

In [2]:
moor = 'L150'
# field_trip = 'rs19'

## Specify the events

In [3]:
wav_tx = get_nliw_times()

In [4]:
# Set key data directories
nc_dir = r'/mnt/share/Experiments/rs19/archive_FV00/Signature 1000/'
nc_files = glob.glob(os.path.join(nc_dir, f'*{moor}*Signature*.nc'))

# Files need natural sorting!!!
nc_files = natsort.natsorted(nc_files)
# nc_files

t_bef = np.timedelta64(0,'m')
t_aft = np.timedelta64(40,'m')

sig_fil = get_sigfiles(wav_tx, t_bef, t_aft, nc_files)
sig_fil

['[RS2019]_[RS19_REC]_[L150]_[28]_[Signature 1000]_[100460]_[13of59].nc',
 '[RS2019]_[RS19_REC]_[L150]_[28]_[Signature 1000]_[100460]_[14of59].nc',
 '[RS2019]_[RS19_REC]_[L150]_[28]_[Signature 1000]_[100460]_[15of59].nc',
 ['[RS2019]_[RS19_REC]_[L150]_[28]_[Signature 1000]_[100460]_[15of59].nc',
  '[RS2019]_[RS19_REC]_[L150]_[28]_[Signature 1000]_[100460]_[16of59].nc'],
 '[RS2019]_[RS19_REC]_[L150]_[28]_[Signature 1000]_[100460]_[16of59].nc',
 '[RS2019]_[RS19_REC]_[L150]_[28]_[Signature 1000]_[100460]_[17of59].nc',
 '[RS2019]_[RS19_REC]_[L150]_[28]_[Signature 1000]_[100460]_[18of59].nc',
 '[RS2019]_[RS19_REC]_[L150]_[28]_[Signature 1000]_[100460]_[19of59].nc',
 ['[RS2019]_[RS19_REC]_[L150]_[28]_[Signature 1000]_[100460]_[19of59].nc',
  '[RS2019]_[RS19_REC]_[L150]_[28]_[Signature 1000]_[100460]_[20of59].nc'],
 '[RS2019]_[RS19_REC]_[L150]_[28]_[Signature 1000]_[100460]_[20of59].nc',
 '[RS2019]_[RS19_REC]_[L150]_[28]_[Signature 1000]_[100460]_[21of59].nc']

In [5]:
save_dir = r'/mnt/c/Users/00099894/Large_Data/RS2019'
save_dir = os.path.join(save_dir, 'nliw')

In [6]:
sig_full = os.path.join(nc_dir, sig_fil[0])
ds1 = xr.open_dataset(sig_full)
ds1

<xarray.Dataset>
Dimensions:                   (beam: 5, height: 254, time: 204479,
                               cartesian_axes: 3)
Coordinates:
  * beam                      (beam) int32 1 2 3 4 5
  * cartesian_axes            (cartesian_axes) int32 1 2 3
  * height                    (height) float64 0.1 0.13 0.16 ... 7.63 7.66 7.69
  * time                      (time) datetime64[ns] 2019-03-12T16:35:59.06340...
    lat_nom                   float64 ...
    lon_nom                   float64 ...
    z_nom                     float64 ...
Data variables: (12/21)
    vel_dolfyn                (beam, height, time) float32 ...
    vel_enu                   (cartesian_axes, height, time) float32 ...
    vel_xyz                   (cartesian_axes, height, time) float32 ...
    four_beam_error_velocity  (height, time) float64 ...
    pressure                  (time) float32 ...
    temperature               (time) float32 ...
    ...                        ...
    qc_velocity               (beam, height, time) float32 ...
    qc_velocity3              (cartesian_axes, height, time) float64 ...
    qc_temperature            (time) float32 ...
    qc_pressure               (time) float32 ...
    qc_compass                (time) float32 ...
    qc_tilt                   (time) float32 ...
Attributes: (12/33)
    title:                           Measured data from a Nortek Signature
    institution:                     The University of Western Australia
    institution_division:            Ocean Dynamics
    source:                          Nortek Signature ADCP
    project:                         RS2019
    history:                         
    ...                              ...
    nominal_site_depth:              150.0
    pressure_sensor_height_asb:      
    nominal_instrument_height_asb:   0.29
    nominal_instrument_orientation:  
    timezone:                        UTC
    is_profile_data:                 0

: 

In [7]:
# Loop through files
for ixll, (file, wave) in enumerate(zip(sig_fil, wav_tx)):
    print(ixll)
    if ixll>-1:
    
        # Load Signature file
        if len(file) == 2:
            sig_full = os.path.join(nc_dir, file[0])
            ds1 = xr.open_dataset(sig_full)
            s_ix1 = (ds1['time'].values >= wave - t_bef) &\
                    (ds1['time'].values <= wave + t_aft)
            xr.Dataset.close(ds1)

            sig_full = os.path.join(nc_dir, file[1])
            ds2 = xr.open_dataset(sig_full)
            s_ix2 = (ds2['time'].values >= wave - t_bef) &\
                    (ds2['time'].values <= wave + t_aft)
            xr.Dataset.close(ds2)

            if (len(ds2.time.isel(time=s_ix2)) > 0):
                print('Concat')
                ds = xr.concat([ds1['echo'].sel(time=s_ix1), ds2['echo'].sel(time=s_ix2)], dim='time')
                # corr = xr.concat([ds1['corr'].sel(time=s_ix1, beam=5), ds2['corr'].sel(time=s_ix2, beam=5)], dim='time')
                del ds1, ds2
            else:
                ds = ds1['echo'].sel(time=s_ix1)
                # corr = ds1['corr'].sel(time=s_ix1, beam=5)
                del ds1
        else:
            sig_full = os.path.join(nc_dir, file)
            dsf = xr.open_dataset(sig_full)

            ## Extract the event
            s_ix = (dsf['time'].values >= wave - t_bef) &\
                    (dsf['time'].values <= wave + t_aft)
        
            ds = dsf['echo'].sel(time=s_ix)
            # corr = dsf['corr'].sel(time=s_ix)
            xr.Dataset.close(dsf)
            del dsf

        # # Save a figure with all 5 beams
        fig, ax = vert_stack(5)
        for i in range(len(ds.beam)):
            ds.sel(beam=i+1).plot(ax=ax[i], cmap=cm.cm.turbid, cbar_kwargs={'pad':0.01})
            ax[i].set_ylabel('Beam ' + str(i+1))
        basic_ts(ds.time.values, ax)
        # saveclose(os.path.join(fig_dir, f"Event_{iii+1}_beams"), fig)

        # Save as netcdf
        ds.to_netcdf(path=os.path.join(save_dir, 'NLIW_' + str(ixll+1) + '_5beam.nc'))
        print('Saved successfully')
        del ds

0
Saved successfully
1
Saved successfully
2
Saved successfully
3
Concat
Saved successfully
4
Saved successfully
5
Saved successfully
6
Saved successfully
7
Saved successfully
8
Concat
Saved successfully
9
Saved successfully
10
Saved successfully
